In [ ]:
import plotly
import json
import requests
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from urllib.request import urlopen
from nowcasting_forecast.database.models import Forecast, ManyForecasts

URL = 'xxx.com'

In [ ]:
# Get GSP boundaries

r = requests.get(URL+'/v0/forecasts/GB/pv/gsp_boundaries/')
d = json.loads(r.json())
boundaries = gpd.GeoDataFrame.from_features(d["features"])


In [ ]:
# get forecasts from API
fileobj = urlopen(URL+'/v0/forecasts/GB/pv/gsp/') 
d = json.loads(fileobj.read())
forecasts = ManyForecasts(**d)


In [ ]:
# format predictions
times = [t.target_time for t in forecasts.forecasts[1].forecast_values]
N_forecast_horizons = len(times)


results = []
for i in range(N_forecast_horizons):
    predictions = {f.location.gsp_id:f.forecast_values[i].expected_power_generation_megawatts for f in forecasts.forecasts}
    predictions_df = pd.DataFrame(list(predictions.items()),columns=['gsp_id','value'])

    boundaries_and_results = boundaries.join(predictions_df, on=['gsp_id'], rsuffix='_r')
    results.append(boundaries_and_results)



In [ ]:
# make frames for plot

boundaries_and_results = boundaries_and_results[~boundaries_and_results.RegionID.isna()]

def make_trace(shapes_dict,boundaries_and_results):
    return go.Choroplethmapbox(
        geojson=shapes_dict, 
        locations=boundaries_and_results.index, 
        z=boundaries_and_results.value, colorscale="Viridis",
        zmax=30,
        zmin=0
    )

# make shape dict for plotting
shapes_dict = json.loads(boundaries_and_results.to_json())

# make annimation
frames = []
for i in range(N_forecast_horizons):
    
    boundaries_and_results = results[i]
    
    frames.append(
        go.Frame(data=[make_trace(shapes_dict,boundaries_and_results)], 
                 layout=go.Layout(title=str(times[i])), 
                 name=f"frame{i+1}")
    )

In [ ]:
# make plot layout options

# make slider dictionary
sliders = [
    dict(
        steps=[
            dict(
                method="animate",
                args=[
                    [f"frame{k+1}"],
                    dict(
                        mode="immediate",
                        frame=dict(duration=600, redraw=True),
                        transition=dict(duration=200),
                    ),
                ],
                label="{}".format(times[k]),
            )
            for k in range(0, len(frames))
        ],
        transition=dict(duration=100),
        x=0,
        y=0,
        currentvalue=dict(font=dict(size=12), visible=True, xanchor="center"),
        len=1.0,
    )
]

# start layout object
layout = go.Layout(
    mapbox_style="carto-positron", mapbox_zoom=6, mapbox_center={"lat": 55, "lon": 0}
)

# add play button and slider 
layout.update(
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            y=0,
            x=0,
            xanchor="left",
            pad=dict(t=5, r=10),
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        dict(
                            frame=dict(duration=600, redraw=True),
                            transition=dict(duration=200),
                            fromcurrent=True,
                            mode="immediate",
                        ),
                    ],
                )
            ],
        )
    ],
    sliders=sliders,
)

In [ ]:
# plot

# plot it
fig = go.Figure(
    frames=frames,
    layout=layout,
    data=make_trace(shapes_dict, results[0])
)


fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=4, mapbox_center={"lat": 55, "lon": 0})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 30})
fig.update_layout(title=times[0].isoformat())
fig.show()
